In [1]:
import cv2
import matplotlib.pyplot as plt
from matplotlib import image
import numpy as np
import os
import re
from scipy import spatial
from scipy import signal
from scipy import ndimage
from scipy import optimize

%matplotlib qt

In [13]:
# https://learnopencv.com/camera-calibration-using-opencv/
CHECKERBOARD = (6,9)
objpoints = []
imgpoints = []
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

im = image.imread('far.jpeg')
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

In [14]:
ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

In [15]:
print(ret)

True


In [16]:
if ret == True:
    objpoints.append(objp)
    corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
    imgpoints.append(corners2)
    im = cv2.drawChessboardCorners(im, CHECKERBOARD, corners2, ret)
plt.imshow(im)
ax = plt.gca()
ax.set_aspect('equal')

In [17]:
h,w = im.shape[:2]
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
optimal_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(
    mtx,
    dist, 
    (w,h), 
    1, 
    (w,h)
)
undistorted_image = cv2.undistort(
    im,
    mtx,
    dist,
    None, 
    optimal_camera_matrix
)
plt.imshow(undistorted_image)
ax = plt.gca()
ax.set_aspect('equal')

In [18]:
def xcorr_prep(im):
    im_corr = np.sum(im.astype(np.float64), axis=2) # ensure single channel
    im_corr = np.max(im_corr) - im_corr # invert value
    im_corr /= im_corr.max() # normalize
    im_corr -= im_corr.mean() # detrend
    return im_corr

In [20]:
plt.close('all')
tmp = np.rot90(image.imread('raft_crop.jpeg'))
stride = 1
a = xcorr_prep(im)[::stride,::stride]
b = xcorr_prep(tmp)[::stride,::stride]
f1 = plt.figure()
plt.imshow(a)
f2 = plt.figure()
plt.imshow(b)
xcorr = signal.fftconvolve(
    a,
    b[::-1, ::-1],
    mode='valid'
)
f3 = plt.figure()
plt.imshow(xcorr)
y,x = np.unravel_index(np.argmax(xcorr), xcorr.shape)
xfound = x + tmp.shape[0] / 2
yfound = y + tmp.shape[1] / 2
ax = f1.get_axes()[0]
ax.scatter(xfound, yfound)
